### 1. 安装HuggingFace 并下载模型到本地

In [ ]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

In [12]:
!rm -rf ./LLM_codellama_model

In [1]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_codellama_model")
local_model_path.mkdir(exist_ok=True)
model_name = "codellama/CodeLlama-13b-hf"
commit_hash = "9e3ca34b26ace9f0c4e098f61f6fa8e9d89dc2c2"

In [14]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

'LLM_codellama_model/models--codellama--CodeLlama-13b-hf/snapshots/9e3ca34b26ace9f0c4e098f61f6fa8e9d89dc2c2'

### 2. 把模型拷贝到S3为后续部署做准备

In [2]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
s3_model_prefix = "LLM-RAG/workshop/LLM_codellama_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_codellama_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_codellama_deploy_code
model_snapshot_path: LLM_codellama_model/models--codellama--CodeLlama-13b-hf/snapshots/9e3ca34b26ace9f0c4e098f61f6fa8e9d89dc2c2


In [17]:
!aws s3 rm --recursive s3://{bucket}/{s3_model_prefix}
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/README.md
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/.gitattributes
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/LICENSE
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/config.json
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/USE_POLICY.md
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/generation_config.json
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/pytorch_model.bin.index.json
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/special_tokens_map.json
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/model.safetensors.index.json
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/tokeni

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [4]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.23.0-deepspeed0.9.5-cu118"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [5]:
!mkdir -p LLM_codellama_deploy_code

In [24]:
%%writefile LLM_codellama_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import deepspeed


def load_model(properties):
    tensor_parallel_degree = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location,trust_remote_code=True)
    
    model = AutoModelForCausalLM.from_pretrained(  
    pretrained_model_name_or_path=model_location,  
    device_map="auto",  
    trust_remote_code=True,
    low_cpu_mem_usage=True,    
    torch_dtype=torch.float16)
    
    #model = deepspeed.init_inference(model,
    #      tensor_parallel={"tp_size": tensor_parallel_degree},
    #      dtype=model.dtype,
    #      replace_method='auto',
    #      replace_with_kernel_inject=True)
    #
    #local_rank = int(os.getenv("LOCAL_RANK", "0"))
    #generator = pipeline(
    #    task="text-generation", model=model, tokenizer=tokenizer, device=local_rank, use_cache=True
    #)
    ### for accelerate ####
    generator = transformers.pipeline(task="text-generation", model=model, tokenizer=tokenizer, device_map="auto", use_cache=True)
    return model, generator, tokenizer


pipeline = None
tokenizer = None
model = None


def handle(inputs: Input):
    global model,pipeline, tokenizer
    if not pipeline:
        model, pipeline, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()

    input_sentences = data["inputs"]
    params = data["parameters"]
    input_ids = tokenizer.encode(input_sentences, return_tensors='pt').to('cuda')

    with torch.no_grad():  
       output_ids = model.generate(  
            input_ids,
            **params)

    response=tokenizer.decode(output_ids[0], skip_special_tokens=True)
    result = {"outputs": response}
    return Output().add_as_json(result)

Overwriting LLM_codellama_deploy_code/model.py


In [25]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

option.s3url ==> s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [26]:
%%writefile LLM_codellama_deploy_code/serving.properties
engine=Python
#engine=DeepSpeed
option.entryPoint=model.py
option.load_in_8bit=TRUE
option.tensor_parallel_degree=4
option.s3url=s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_model/

Overwriting LLM_codellama_deploy_code/serving.properties


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

如果是中国区建议添加国内的pip镜像,如下代码所示

%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1


In [27]:
%%writefile LLM_codellama_deploy_code/requirements.txt
#-i https://pypi.tuna.tsinghua.edu.cn/simple
#transformers==4.30.2
git+https://github.com/huggingface/transformers

Overwriting LLM_codellama_deploy_code/requirements.txt


In [28]:
!rm model.tar.gz
!cd LLM_codellama_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_codellama_deploy_code

LLM_codellama_deploy_code/
LLM_codellama_deploy_code/requirements.txt
LLM_codellama_deploy_code/model.py
LLM_codellama_deploy_code/serving.properties


In [29]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_codellama_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [30]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"codellame") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

codellame-2023-09-22-09-25-02-063
Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-west-2:687912291502:model/codellame-2023-09-22-09-25-02-063


In [31]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.12xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            #"ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:687912291502:endpoint-config/codellame-2023-09-22-09-25-02-063-config',
 'ResponseMetadata': {'RequestId': '95645bfe-b578-4ff2-92b5-2411113516b8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '95645bfe-b578-4ff2-92b5-2411113516b8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '121',
   'date': 'Fri, 22 Sep 2023 09:25:04 GMT'},
  'RetryAttempts': 0}}

In [32]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:687912291502:endpoint/codellame-2023-09-22-09-25-02-063-endpoint


#### 持续检测模型部署进度

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:687912291502:endpoint/codellame-2023-09-22-09-25-02-063-endpoint
Status: InService


### 5. 模型测试

In [ ]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_new_tokens": 300,
  "temperature": 0.5,
  #"num_beams": 1, # >1可能会报错，"probability tensor contains either inf, nan or element < 0"； 即使remove_invalid_values=True也不能解决
  #"do_sample": False,
  #"top_p": 0.7,
  #"logits_processor" : None,
  # "remove_invalid_values" : True
}

CPU times: user 3.98 ms, sys: 128 µs, total: 4.11 ms
Wall time: 3.76 ms


In [36]:
sql_prompt= """
You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 3 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks () to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
CREATE TABLE ads_bi_quality_monitor_shipping_detail (
  shipping_order_code varchar(100)  COMMENT '派车单编码',
  license_plate varchar(100)  COMMENT '车牌号',
  truck_type varchar(50)  COMMENT '车辆类型',
  tenant_id varchar(100)  COMMENT '租户编码',
  tenant_name varchar(200)  COMMENT '租户名称',
  father_company_code varchar(100)  COMMENT '分子公司编码',
  father_company_name varchar(200)  COMMENT '分子公司名称',
  father_company_short_name varchar(200)  COMMENT '分子公司简称',
  start_transport_time varchar(50)  COMMENT '运输出发时间',
  signing_time varchar(50)  COMMENT '派车单签收时间',
  plan_start_time varchar(50)  COMMENT '派车单计划取货时间',
  plan_end_time varchar(50)  COMMENT '派车单计划送达时间',
  frist_fence_time varchar(50)  COMMENT '第一次碰撞装货地电子围栏时间',
  leave_load_station_2km_time varchar(50)  COMMENT '离开最后一个装货地电子围栏2km的时间',
  frist_arrive_unload_time varchar(50)  COMMENT '首次到达卸货点电子围栏时间',
  transport_type varchar(10)  COMMENT '运输类型(干线/城配)',
  plan_mileage varchar(50)  COMMENT '计划里程',
  driver_accept_time varchar(50)  COMMENT '司机接单时间',
  driver_name varchar(50)  COMMENT '司机姓名',
  driver_phone varchar(50)  COMMENT '司机电话',
  driver_type varchar(10)  COMMENT '司机类型',
  transport_tenant_id varchar(100)  COMMENT '运力承运商id',
  transport_tenant_name varchar(200)  COMMENT '运力承运商名称',
  warm_area varchar(50)  COMMENT '温区信息',
  waybill_count int DEFAULT NULL COMMENT '运单数量',
  gps_device_list varchar(100)  COMMENT 'gps设备',
  gps_report_dot_num int DEFAULT NULL COMMENT 'gps上报点数',
  temp_substandard_min decimal(20,8) DEFAULT NULL COMMENT '温度不达标时长(分钟)',
  temp_substandard_min_n12 decimal(20,8) DEFAULT NULL COMMENT '温度不达标时长(分钟)_-12',
  temp_substandard_min_n16 decimal(20,8) DEFAULT NULL COMMENT '温度不达标时长(分钟)_-16',
  shipping_order_transport_min decimal(20,8) DEFAULT NULL COMMENT '派车单运输总时长(分钟)',
  shipping_order_temp_standard_rate decimal(20,8) DEFAULT NULL COMMENT '派车单温度达标率',
  shipping_order_temp_standard_rate_n12 decimal(20,8) DEFAULT NULL COMMENT '派车单温度达标率_-12',
  shipping_order_temp_standard_rate_n16 decimal(20,8) DEFAULT NULL COMMENT '派车单温度达标率_-16',
  dep_fence_match_num int DEFAULT NULL COMMENT '出发地电子围栏匹配数量',
  dep_total_num int DEFAULT NULL COMMENT '出发地总数量',
  dep_fence_match_rate decimal(20,8) DEFAULT NULL COMMENT '出发地电子围栏匹配率',
  des_fence_match_num int DEFAULT NULL COMMENT '目的地电子围栏匹配数量',
  des_total_num int DEFAULT NULL COMMENT '目的地总数量',
  des_fence_match_rate decimal(20,8) DEFAULT NULL COMMENT '目的地电子围栏匹配率',
  exception_shipping_order_type varchar(10)  COMMENT '异常派车单情况,0-非异常',
  settlement_tenant_id varchar(100)  COMMENT '结算主体租户id',
  settlement_tenant_name varchar(100)  COMMENT '结算主体租户名称',
  truck_ownership varchar(100)  COMMENT '车辆所有权(自有/外请/临时)',
  load_finish_time varchar(50)  COMMENT '派车单点击装货完成时间',
  last_leave_unload_time varchar(50)  COMMENT '最后离开卸货点时间',
  temp_right_tag varchar(20)  COMMENT '温度是否合格(合格/不合格/不参与评估)',
  warm_area_type varchar(20)  COMMENT '温区类型。常温/冷链/自定义区间',
  source_dt varchar(20)  COMMENT '派车单来源的增量表dt',
  cur_shipping_odr_cust text CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci COMMENT '当前派车单对应的客户',
  is_app_operation varchar(10)  COMMENT '是否APP操作卡控',
  leave_load_station_time_app varchar(50)  COMMENT '离开第一个装货地时间_app',
  frist_arrive_unload_time_app varchar(50)  COMMENT '到达第一个卸货点时间_app',
  last_arrive_unload_time_app varchar(50)  COMMENT '到达最后一个卸货点时间_app',
  temp_calc_start_time varchar(50)  COMMENT '温度计算开始时间',
  temp_calc_end_time varchar(50)  COMMENT '温度计算结束时间',
  exception_shipping_order_type_desc varchar(100)  COMMENT '异常派车单情况描述',
  root_shipping_odr_cust text CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci COMMENT '根派车单对应的客户',
  is_use_custom_temp_range varchar(10)  COMMENT '是否使用自定义温度范围',
  temp_eval_lowest int DEFAULT NULL COMMENT '温度考核最低值',
  temp_eval_highest int DEFAULT NULL COMMENT '温度考核最高值',
  is_gps_cover varchar(10)  COMMENT 'GPS是否覆盖',
  shipping_create_time varchar(50)  COMMENT '派车单创建时间',
  gps_device_list_plan varchar(100)  COMMENT 'GPS设备_预估',
  is_gps_cover_plan varchar(10)  COMMENT 'GPS是否覆盖_预估',
  first_station_is_on_time varchar(10)  COMMENT '首店是否准时',
  is_many_warm varchar(10)  COMMENT '是否多温区',
  many_temp_standard_rate decimal(20,8) DEFAULT NULL COMMENT '多温区温度达标率',
  lc_standard_ratio decimal(20,8) DEFAULT NULL COMMENT '冷藏温度达标率',
  ld_standard_ratio decimal(20,8) DEFAULT NULL COMMENT '冷冻温度达标率',
  probe_warm_list text CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci COMMENT '探头温区',
  KEY idx_shipping_code (shipping_order_code),
  KEY idx_sdt (source_dt)
) ENGINE=InnoDB  COMMENT='BI看板_品质监控_派车单明细';

Question: 最近一个月温度合格的派车单数量
"""
endpoint_name="codellame-2023-09-22-09-25-02-063-endpoint"
response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": sql_prompt,
                #"inputs": "write a python code for hello world",
                
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

'{\n  "outputs":"\\nYou are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 3 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.\\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks () to denote them as delimited identifiers.\\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\\nPay attention to use CURDATE() function to get the current date, if the question involves \\"today\\".\\n\\nUse the following format:\\n\\nQuestion: Question here\\nSQLQuery: SQ

#### 清除模型Endpoint和config

In [ ]:
!aws sagemaker delete-endpoint --endpoint-name codellama-2023-06-27-06-53-41-986-endpoint